**Mounting My Drive for the files**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset link : https://drive.google.com/file/d/1LLFx4aEELA5PTbVMyARZATrQ5fJON5Pw/view?usp=drive_link

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importing Packages**

In [2]:
import os
import random
import torch
import numpy as np
import pandas as pd
import json
!pip install gradio
import gradio as gr
from torch import nn, optim
import torch.nn.functional as F

**Defining The Neural BPR Model**

In [3]:
# ---------- Model Definition ----------
class NBPR(nn.Module):
    def __init__(self, num_users, num_items, emb_dim=20, hidden_dims=[64,32], reg=1e-4):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        layers = []
        input_dim = emb_dim * 2
        for h in hidden_dims:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU())
            input_dim = h
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)
        self.reg = reg
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)

    def forward(self, user, item):
        p = self.user_emb(user)
        q = self.item_emb(item)
        x = torch.cat([p, q], dim=1)
        return self.mlp(x).squeeze()

    def bpr_loss(self, u, i, j):
        r_ui = self(u, i)
        r_uj = self(u, j)
        loss = -F.logsigmoid(r_ui - r_uj).mean()
        reg_term = (self.user_emb(u).pow(2).sum() +
                    self.item_emb(i).pow(2).sum() +
                    self.item_emb(j).pow(2).sum())
        for param in self.mlp.parameters():
            reg_term += param.pow(2).sum()
        return loss + self.reg * reg_term


**Getting The Data Ready**

In [4]:
# ---------- Data Preparation ----------
# data.csv location : https://drive.google.com/file/d/1LLFx4aEELA5PTbVMyARZATrQ5fJON5Pw/view?usp=drive_link
def get_song_indices(names, df):
    indices = []
    for name in names:
        name = name.strip().lower()
        matches = df.index[df['name'].str.lower() == name].tolist()
        if matches:
            indices.append(matches[0])
    return list(set(indices))
# If downloading dataset, please replace filepath with data.csv
def load_and_preprocess():
    df = pd.read_csv('/content/drive/MyDrive/data.csv').dropna().reset_index(drop=True)
    return df

**Recommendation System**

In [5]:
# ---------- Modified Recommender System ----------
class RecommenderSystem:
    def __init__(self):
        self.df = load_and_preprocess()
        self.num_items = len(self.df)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def train_on_input(self, input_songs, epochs=100):
        user_pos = get_song_indices(input_songs, self.df)

        # If no valid song was found, return early
        if not user_pos:
            return None

        # Initialize a new model each time
        model = NBPR(1, self.num_items).to(self.device)
        opt = optim.Adam(model.parameters(), lr=1e-3)
        neg_pool = list(set(range(self.num_items)) - set(user_pos))

        # Quick training loop
        for epoch in range(epochs):
            batch_size = min(512, len(user_pos) * 10)  # Adjust batch size based on input
            u = torch.zeros(batch_size, dtype=torch.long, device=self.device)
            i = torch.tensor(np.random.choice(user_pos, batch_size, replace=True), device=self.device)
            j = torch.tensor(np.random.choice(neg_pool, batch_size), device=self.device)

            loss = model.bpr_loss(u, i, j)
            opt.zero_grad()
            loss.backward()
            opt.step()

        return model

    def recommend(self, user_input, temperature=1.0, top_k=20):
        input_songs = [s.strip() for s in user_input.split(",") if s.strip()]

        # Train model on user input
        model = self.train_on_input(input_songs)

        # If no valid songs were found, return empty DataFrame
        if model is None:
            return pd.DataFrame(columns=["Song", "Artist", "Score", "Probability"])

        exclude_indices = get_song_indices(input_songs, self.df)

        # Generate recommendations
        with torch.no_grad():
            u = torch.zeros(self.num_items, dtype=torch.long, device=self.device)
            items = torch.arange(self.num_items, device=self.device)
            scores = model(u, items).cpu().numpy()

        # Apply temperature scaling
        scores[exclude_indices] = -np.inf
        valid_scores = scores.copy()
        valid_scores[exclude_indices] = -np.inf

        # Convert to probabilities using softmax with temperature
        exp_scores = np.exp(valid_scores / temperature)
        probs = exp_scores / exp_scores.sum()

        # Sample without replacement using PyTorch multinomial
        valid_indices = np.where(valid_scores != -np.inf)[0]

        if len(valid_indices) == 0:
            return pd.DataFrame(columns=["Song", "Artist", "Score", "Probability"])

        sampled_indices = torch.multinomial(
            torch.tensor(probs[valid_indices], dtype=torch.float32),
            num_samples=min(top_k, len(valid_indices)),
            replacement=False
        ).numpy()

        # Get final recommendations
        top_indices = valid_indices[sampled_indices]

        results = []
        for idx in top_indices:
            song = self.df.iloc[idx]
            results.append({
                "Song": song['name'],
                "Artist": song['artists'],
                "Score": f"{scores[idx]:.4f}",
                "Probability": f"{probs[idx]:.4f}"
            })

        return pd.DataFrame(results)

**Putting It All Together and Gradio Interface**

In [6]:
if __name__ == "__main__":
    # Initialize recommender system
    rs = RecommenderSystem()

    # Create Gradio interface with controls
    with gr.Blocks(title="Dynamic Song Recommender") as demo:
        gr.Markdown("# 🎵 Dynamic Song Recommender")
        gr.Markdown("Enter songs you like and adjust parameters for diverse recommendations")

        with gr.Row():
            song_input = gr.Textbox(
                label="Your favorite songs (comma separated)",
                placeholder="e.g.: Shape of You, Bohemian Rhapsody"
            )
            with gr.Column():
                temperature = gr.Slider(0.1, 2.0, value=0.5,
                                      label="Diversity Control (Temperature)")
                top_k = gr.Slider(5, 50, value=20, step=1,
                                label="Number of Recommendations")

        recommend_btn = gr.Button("Generate Recommendations")
        output_df = gr.DataFrame(
            headers=["Song", "Artist", "Score", "Probability"],
            label="Recommended Songs"
        )

        recommend_btn.click(
            fn=rs.recommend,
            inputs=[song_input, temperature, top_k],
            outputs=output_df
        )

    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b72f8b0649a0d79670.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
